In [3]:
import pandas as pd
from database.database import Database
from Embedders import get_embedder


db = Database()
db.set_session_resources(optimize_for="query", verbose=True)

Executing query:

                -- SET synchronous_commit = 'off';
                SET maintenance_work_mem = '1GB';
                -- SET random_page_cost = '1.1';
                -- SET parallel_tuple_cost = '0.1';
                -- SET parallel_setup_cost = '1000';
                SET max_parallel_workers = 10;
                SET work_mem = '4.0GB';
                SET max_parallel_workers_per_gather = 10;
                SET effective_cache_size = '64GB';
            


In [4]:
samples = pd.read_json('data/dataset/nontrivial_llm.jsonl', lines=True)
samples.head()

,source_doi,sent_original,sent_no_cit,sent_idx,citation_dois,pubdate,resolved_bibcodes
0,10.1016/j.newar.2024.101694,"Subsequently, Andrews et al. (2017) selected a...","Subsequently, [REF] selected a wide binary can...",58,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
1,10.1016/j.newar.2024.101694,Andrews et al. (2017) investigated how the sep...,[REF] investigated how the separation of their...,61,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
2,10.1016/j.newar.2024.101694,This led Andrews et al. (2017) to conclude tha...,This led [REF] to conclude that most of the pa...,64,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
3,10.1016/j.newar.2024.101694,It may also owe in part to the mass ratio dist...,It may also owe in part to the mass ratio dist...,90,[10.1093/mnras/stz2480],2024-06-01,[2019MNRAS.489.5822E]
4,10.1016/j.newar.2024.101694,Hwang et al. (2022c) used a related method to ...,[REF] used a related method to study the eccen...,110,[10.3847/2041-8213/ac7c70],2024-06-01,[2022ApJ...933L..32H]


In [9]:
example_1 = samples.iloc[0]
print(type(example_1))

query_text = example_1['sent_no_cit']
print(query_text)
print(example_1['pubdate'])

<class 'pandas.core.series.Series'>
Subsequently, [REF] selected a wide binary candidate sample from TGAS with an emphasis on purity, using a Bayesian method to calculate the probability that a given pair of stars is gravitationally bound.
2024-06-01


In [20]:
embedder = get_embedder("BAAI/bge-small-en", device="mps", normalize=True)
query_vector = embedder([query_text])[0]
print(f"Vector shape: {query_vector.shape}")

# Get top 10 most similar docs
results = db.query_vector_column(
    query_vector=query_vector,
    table_name="lib",
    target_column="bge_norm",
    metric="vector_cosine_ops",
    pubdate=example_1['pubdate'],
    top_k=10,
    ef_search=40
)
print(f"Results: {len(results)}")

Vector shape: (384,)
Results: 10


In [21]:
target_dois = set(example_1['citation_dois'])
print(f"Target doi(s): {target_dois}")

retrieved_dois = set([r.doi for r in results])
print(f"Retrieved doi(s): {retrieved_dois}")

print(f"Target in retrieved set?: {target_dois.issubset(retrieved_dois)}")

Target doi(s): {'10.1093/mnras/stx2000'}
Retrieved doi(s): {'10.1093/mnras/stx2000', '10.1086/381483', '10.1093/mnras/stz2480', '10.1111/j.1365-2966.2010.16896.x', '10.1088/0004-637X/738/1/79', '10.1093/mnras/stab323'}
Target in retrieved set?: True


In [22]:
print([results.doi for results in results])

['10.1093/mnras/stab323', '10.1093/mnras/stx2000', '10.1086/381483', '10.1093/mnras/stx2000', '10.1093/mnras/stz2480', '10.1093/mnras/stx2000', '10.1093/mnras/stx2000', '10.1088/0004-637X/738/1/79', '10.1093/mnras/stx2000', '10.1111/j.1365-2966.2010.16896.x']
